In [1]:
from datetime import datetime, timezone, timedelta
import struct
import numpy as np
from typing import NamedTuple
import xarray as xr

In [2]:
fp = 'C:/Users\Ian\projects/acpype/tests/test_data\OPTAAD106_10.33.14.8_2101_20140930T1200_UTC.dat'

In [3]:
with open(fp, 'rb') as bf:
    contents = bytearray(bf.read())

In [4]:
PACKET_REGISTRATION = b'\xff\x00\xff\x00'
LEN_REG = len(PACKET_REGISTRATION)
PAD_BYTE = b'\x00'
PACKET_HEAD = '!4cHBBl7HIBB'
PACKET_TAIL = 'Hx'
LEN_PREDEFINED = struct.calcsize(PACKET_HEAD + PACKET_TAIL)
LEN_CHECK_PAD = 2 + 1
WVL_BYTE_OFFSET = 4 + 2 + 1 + 1 + 1 + 3 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 4 + 1  # See Process Data section in ACS manual.


class ParsedPacket(NamedTuple):
    reg_bytes: tuple
    record_length: int
    packet_type: int
    reserved_1: int
    serial_number_int: int
    a_reference_dark:  int
    pressure_raw:  int
    a_signal_dark:  int
    external_temperature_raw:  int
    internal_temperature_raw:  int
    c_reference_dark:  int
    c_signal_dark:  int
    elapsed_time:  int
    reserved_2:  int
    number_of_wavelength_bins:  int
    c_reference:  tuple
    c_signal:  tuple
    a_reference:  tuple
    a_signal:  tuple
    checksum: int
    
    
def parse_packet(packet):
    [nwvls] = struct.unpack_from('B', packet, offset=WVL_BYTE_OFFSET)
    size_of_remaining_bytes = int(nwvls * 2 * 2)
    if packet[-1] != int.from_bytes(PAD_BYTE):
        descriptor = PACKET_HEAD + f"{size_of_remaining_bytes}H"
    else:
        descriptor = PACKET_HEAD + f"{size_of_remaining_bytes}H" + 'x'
    data = struct.unpack_from(descriptor, packet)
    parsed_packet = ParsedPacket(reg_bytes=data[:LEN_REG],
                                 record_length=data[LEN_REG + 0],
                                 packet_type=data[LEN_REG + 1],
                                 reserved_1=data[LEN_REG + 2],
                                 serial_number_int=data[LEN_REG + 3],
                                 a_reference_dark=data[LEN_REG + 4],
                                 pressure_raw=data[LEN_REG + 5],
                                 a_signal_dark=data[LEN_REG + 6],
                                 external_temperature_raw=data[LEN_REG + 7],
                                 internal_temperature_raw=data[LEN_REG + 8],
                                 c_reference_dark=data[LEN_REG + 9],
                                 c_signal_dark=data[LEN_REG + 10],
                                 elapsed_time=data[LEN_REG + 11],
                                 reserved_2=data[LEN_REG + 12],
                                 number_of_wavelength_bins=data[LEN_REG + 13],
                                 c_reference=data[LEN_REG + 14::4],
                                 c_signal=data[LEN_REG + 15::4],
                                 a_reference=data[LEN_REG + 16::4],
                                 a_signal=data[LEN_REG + 17::4],
                                 checksum=data[LEN_REG + 13 + (data[LEN_REG + 13] * 4)])
    return parsed_packet


In [5]:
parsed_packets = []
old_bytes, sep, remaining_bytes = contents.partition(PACKET_REGISTRATION)
for i in range(3):
    if PACKET_REGISTRATION in remaining_bytes:
        dt = datetime.now()
        packet_data, next_sep, next_loop_bytes = remaining_bytes.partition(PACKET_REGISTRATION)
        packet = sep + packet_data 
        contents = next_sep + next_loop_bytes # Redefine data.
        pad_byte_idx = packet.rfind(PAD_BYTE) # Pad byte index should always represent the last byte in the packet.
        checksum = packet[pad_byte_idx-2:pad_byte_idx]  # The checksum is the two bytes that occur before the pad 
        if np.uint16(sum(packet[:-LEN_CHECK_PAD])) != struct.unpack_from('!H', checksum):
            print('Bad Packet')
            break
        else:
            parsed = parse_packet(packet)
            parsed_packets.append(parsed)
            

In [6]:
parsed_packets

[ParsedPacket(reg_bytes=(b'\xff', b'\x00', b'\xff', b'\x00'), record_length=648, packet_type=5, reserved_1=1, serial_number_int=1392509097, a_reference_dark=466, pressure_raw=0, a_signal_dark=699, external_temperature_raw=40409, internal_temperature_raw=51059, c_reference_dark=477, c_signal_dark=712, elapsed_time=408053953, reserved_2=1, number_of_wavelength_bins=77, c_reference=(419, 492, 567, 650, 745, 849, 964, 1099, 1247, 1415, 1597, 1792, 2002, 2226, 2461, 2715, 2991, 3298, 3618, 3969, 4345, 4749, 5170, 5604, 6052, 6520, 6980, 7439, 7911, 8404, 8931, 9497, 10107, 10744, 11383, 12022, 12636, 13232, 13799, 14350, 14895, 15424, 15895, 16450, 16880, 17279, 17612, 17909, 18150, 18319, 18436, 18477, 18451, 18371, 18227, 18033, 17754, 17436, 17105, 16729, 16313, 15829, 15282, 14680, 14030, 13360, 12696, 12012, 11337, 10665, 10002, 9359, 8719, 8107, 7541, 7011, 6492), c_signal=(641, 745, 838, 936, 1046, 1163, 1293, 1443, 1604, 1791, 1990, 2198, 2421, 2656, 2894, 3157, 3443, 3755, 4091, 44

In [7]:
ncfp = "C:/Users\Ian\projects/acpype/tests/test_data\deployment0001_CE02SHBP-LJ01D-08-OPTAAD106-streamed-optaa_sample_20140925T182900-20141118T095959.nc"
ds = xr.open_dataset(ncfp)

In [8]:
test = ds.where(ds.elapsed_run_time == 408053953, drop = True)

In [9]:
nc_parse = test.a_signal_counts

In [10]:
my_parse = parsed_packets[0]

In [11]:
for i in range(len(my_parse.a_signal)):
    print(my_parse.a_signal[i],nc_parse[0].values[i], my_parse.a_signal[i] ==nc_parse[0].values[i])

436 436.0 True
539 539.0 True
631 631.0 True
732 732.0 True
842 842.0 True
960 960.0 True
1091 1091.0 True
1244 1244.0 True
1409 1409.0 True
1603 1603.0 True
1812 1812.0 True
2037 2037.0 True
2280 2280.0 True
2541 2541.0 True
2812 2812.0 True
3110 3110.0 True
3442 3442.0 True
3803 3803.0 True
4198 4198.0 True
4622 4622.0 True
5081 5081.0 True
5578 5578.0 True
6086 6086.0 True
6624 6624.0 True
7189 7189.0 True
7755 7755.0 True
8330 8330.0 True
8909 8909.0 True
9512 9512.0 True
10154 10154.0 True
10841 10841.0 True
11574 11574.0 True
12372 12372.0 True
13200 13200.0 True
14040 14040.0 True
14885 14885.0 True
15705 15705.0 True
16504 16504.0 True
17271 17271.0 True
17999 17999.0 True
18723 18723.0 True
19398 19398.0 True
19976 19976.0 True
20668 20668.0 True
21129 21129.0 True
21530 21530.0 True
21881 21881.0 True
22219 22219.0 True
22508 22508.0 True
22738 22738.0 True
22896 22896.0 True
22963 22963.0 True
22956 22956.0 True
22869 22869.0 True
22702 22702.0 True
22450 22450.0 True
22066 